<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/avalia_modelos_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pacotes
import os
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy import stats
import pickle
from PIL import Image, ImageDraw, ImageFont
import sklearn
#
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.backend import expand_dims, repeat_elements
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import ConvLSTM2D, Bidirectional, Input, Permute, Reshape, TimeDistributed, Lambda, Conv2D, Conv3D, Dense, Dropout, Normalization, BatchNormalization

In [2]:

from google.colab import drive
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [3]:
# DataGenerator de https://github.com/mauriciodev/tec_forecast/blob/main/examples/tec_forecast_tf2_lab.ipynb
# split_sequence alterada para levar em conta que estamos trabalhando com patches na linha 71
# e também o self.list_IDs da linha 9
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, x, batch_size, nstepsin=4, nstepsout=1, shuffle=True,training=True, sample_rate=12, val_split=0, validation=False, random_state=23, test_set = 0):
        'Initialization'
        super().__init__()
        self.list_IDs=range(0,len(x)-(nstepsout-1+nstepsin)*100,sample_rate) #store the index to allow shuffling
        if val_split>0:
            datasplit=train_test_split(self.list_IDs,random_state=random_state, test_size=val_split)
            if validation==False: #training generator
                self.list_IDs=datasplit[0]
            else:
                self.list_IDs=datasplit[1]

        # Mexendo na seleção de amostras para treinamento - queremos manter mais amostras que apresentem variação
        if test_set == 0:
          id_keep = []
          for current_id in self.list_IDs:
            l_test = x[current_id:current_id+(2*100):100]
            if np.any(l_test[0] == 1):
              if np.any(l_test[1] == 1):
                id_keep.append(current_id)
          self.list_IDs = id_keep

        self.x=x
        self.nstepsin=nstepsin
        self.nstepsout=nstepsout
        self.batch_size=batch_size
        self.dim=x[0].shape
        self.shuffle=shuffle
        self.training=training
        self.on_epoch_end()

    def count(self):
        'Returns the number of samples'
        return len(self.list_IDs)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        if self.training:
            return X, y
        else:
            return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        Y = []
        # Generate data
        for ID in list_IDs_temp:
            x,y=self.split_sequence(ID)
            X.append(x)
            Y.append(y) # <- alteração aqui
        return np.array(X), np.array(Y).astype(np.int32)

    def split_sequence(self, i):
        # find the end of this pattern
        end_ix = i + self.nstepsin*100
        # check if we are beyond the sequence
        #if (end_ix + (self.nstepsout)*100) > len(self.x):
        #    return None,None
        # gather input and output parts of the pattern
        seq_x, seq_y = self.x[i:end_ix:100], self.x[end_ix:(end_ix+(self.nstepsout*100)):100]
        return seq_x,seq_y

    def asArray(self):
        return self.__data_generation(self.list_IDs)

In [4]:
with open("/content/drive/My Drive/projeto_DL/dataset_tratado/test_npy.pkl", 'rb') as handle:
  test_npy = pickle.load(handle)

In [5]:
nstepsin=15
nstepsout=1
time_stride=1
batch_size=1
random_seed=1
sample_rate_in=1
sample_rate_out=1

In [6]:
test_set=DataGenerator(test_npy, batch_size=batch_size, nstepsin=nstepsin, nstepsout=nstepsout, shuffle=False, val_split=0,
                       validation=False, random_state=random_seed, sample_rate=sample_rate_out, test_set = 1)

In [7]:
print("X batches:",test_set[0][0].shape)
print("Y batches:",test_set[0][1].shape)
print("Training sSamples: ", test_set.count())

X batches: (1, 15, 256, 256, 1)
Y batches: (1, 1, 256, 256, 1)
Training sSamples:  3700


In [8]:
model = tf.keras.saving.load_model("/content/drive/My Drive/projeto_DL/modelo_01.h5")

In [9]:
prevs = []
real = []

for i in range(test_set.count()):
  #
  v_interno = np.expand_dims(model.predict(test_set[i][0]).argmax(axis=-1), axis=-1)
  v_interno = v_interno[0,0,:,:,0]
  prevs.append(v_interno.copy())
  #
  v2_interno = test_set[i][1]
  v2_interno = v2_interno[0,0,:,:,0]
  real.append(v2_interno.copy())
  #
  if (i % 50) == 0:
    print(i)


1/1 [==============================] - 9s 9s/step
0
1/1 [==============================] - 0s 54ms/step
50
1/1 [==============================] - 0s 34ms/step
100
1/1 [==============================] - 0s 37ms/step
150
1/1 [==============================] - 0s 42ms/step
200
1/1 [==============================] - 0s 33ms/step
250
1/1 [==============================] - 0s 41ms/step
300
1/1 [==============================] - 0s 44ms/step
350
1/1 [==============================] - 0s 33ms/step
400
1/1 [==============================] - 0s 32ms/step
450
1/1 [==============================] - 0s 49ms/step
500
1/1 [==============================] - 0s 31ms/step
550
1/1 [==============================] - 0s 34ms/step
600
1/1 [==============================] - 0s 35ms/step
650
1/1 [==============================] - 0s 31ms/step
700
1/1 [==============================] - 0s 36ms/step
750
1/1 [==============================] - 0s 30ms/step
800
1/1 [==============================] - 0s 31ms/step
8

In [10]:
mt_ = np.zeros((3,3), int)

for i in range(len(real)):
  mt_ = np.add(mt_,
               sklearn.metrics.confusion_matrix(
                  np.reshape(real[i], -1), np.reshape(prevs[i], -1),
                  labels = [0,1,2])
  )


In [11]:
mt_

array([[136819431,  13430249,     29551],
       [    85081,      9594,        28],
       [ 17799777,  28357417,  45952072]])

In [12]:
mt2_ = np.zeros((3,3), int)

for i in range(len(real)):
  idx_aval = np.reshape(real[i], -1) != 2
  mt2_ = np.add(mt2_,
               sklearn.metrics.confusion_matrix(
                  np.reshape(real[i], -1)[idx_aval], np.reshape(prevs[i], -1)[idx_aval],
                  labels = [0,1,2])
  )


In [13]:
mt2_

array([[136819431,  13430249,     29551],
       [    85081,      9594,        28],
       [        0,         0,         0]])

In [29]:
np.divide(mt2_, np.array([[1586, 1586, 1586], [1, 1, 1], [1, 1, 1]])).astype(int)

array([[86266,  8468,    18],
       [85081,  9594,    28],
       [    0,     0,     0]])